#### Function to Splite Your Data to Three : CSV and JSON and DB_file :

In [1]:
import pandas as pd
import pyodbc 
import json

In [1]:
# add you persentage data :
# p_json : for json 
# p-Db : for Base de donnee
# csv : is the keep pesontage .

def data_spliter(p_json,p_DB,csv_file_name,file_json_name_part,driver,server,database,table_name,file_Csv_name_part):

    # parameter cheking :
    if p_json + p_DB <= 100 and '.csv' in csv_file_name :
        # read csv :
        df = pd.read_csv(csv_file_name)

        # drop the NuLL value :
        df.dropna(axis=1, inplace=True)

        # do random for row in DataFrame :
        df_random_order = df.sample(frac=1, random_state=42)

        # list as names of all columns :
        columns = list(df_random_order.columns)

        # calculation of row in such type csv json db :
        p_json = int(df_random_order.shape[0] * (p_json  * 0.01))
        p_DB = int(df_random_order.shape[0] * (p_DB  * 0.01))
        p_csv = df_random_order.shape[0] - (p_json + p_DB)
        #----------------------------------------------------------------
        # creating file json :
        # --------------------
        # all row for json file :
        json_data_list = df_random_order[0:p_json].to_dict(orient='records')

        # create file json :
        with open(f'{file_json_name_part}_{p_json}_row.json', 'w') as json_file :
            json.dump(json_data_list, json_file, indent=4)

        #----------------------------------------------------------------
        # creating Data Base :
        #---------------------
            # part of data for Db :
            db_data = df_random_order[p_json + 1:p_json + 1 + p_DB]

            # Créer un connexion dans le database :
            driver = 'SQL Server'

            conn = pyodbc.connect(f'''
                                    DRIVER={driver};
                                    SERVER={server};
                                    DATABASE={database};
                                    Trusted_Connection=yes
                                ''')
            cursor = conn.cursor()

            # creating Table :
            query = f"CREATE TABLE [{table_name}_{p_DB}_row] (\n"

            for col in columns:
                query += f"    {col} VARCHAR(MAX),\n"

            query = query[:-2] + "\n);"

            cursor.execute(query)
            cursor.commit()

            # Create the SQL INSERT query template
            insert_query = f"INSERT INTO [{table_name}_{p_DB}_row] ({', '.join(columns)}) VALUES ({', '.join(['?'] * len(columns))})"

            # Iterate through the DataFrame using iterrows() and execute the INSERT queries
            for _, row in db_data.iterrows():
                var = []
                for i in columns :
                    var.append(row[i])
                cursor.execute(insert_query, var)
            cursor.commit()
        #----------------------------------------------------------------
        # create csv file :
        #---------------------
        # part of data comme csv :
        if p_json + p_DB != 100 :
            Csv_data = df_random_order[p_json + 1 + p_DB + 1:]

            # create file csv :
            df.to_csv(f'{file_Csv_name_part}_{Csv_data.shape[0]}_row.csv', index=False, header=False)
        #----------------------------------------------------------------
    else :
        print('param error')

In [29]:
data_spliter(58,15,'books.csv','lovelove','my_jsonfile','LAPTOP-6D8J0VI4\SQLEXPRESS','xxxx','loooooo','my_csv_tes')